In [4]:
#!pip install modin[all]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import modin.pandas as pd
import numpy as np

df=pd.read_csv("/content/drive/MyDrive/kurly_hackathon/data/2019-Nov.csv") # 2019년 11월 데이터

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(deploy_ray_func pid=1071) tcmalloc: large alloc 2251694080 bytes == 0x360c000 @  0x7f7acf1e61e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f7acbebfe02 0x7f7acbf53db6 0x7f7acbec6306 0x7f7acc03a6ab 0x7f7acbf9b88f 0x7f7acc0627d3 0x7f7acc06372a 0x7f7acc07518e 0x7f7acc04e530 0x7f7acc272f06 0x7f7acc21fa3e 0x7f7acc21fc96 0x7f7acc6b6cab 0x7f7acc6b7ee1
(deploy_ray_func pid=1069) tcmalloc: large alloc 2251694080 bytes == 0x44f0000 @  0x7f798d9ca1e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f798a6a3e02 0x7f798a737db6 0x7f798a6aa306 0x7f798a81e6ab 0x7f798a77f88f 0x7f798a8467d3 0x7f798a84772a 0x7f798a85918e 0x7f798a832530 0x7f798aa56f06 0x7f798aa03a3e 0x7f798aa03c96 0x7f798

In [7]:
df1=pd.read_csv("/content/drive/MyDrive/kurly_hackathon/data/2019-Oct.csv") # 2019년 10월 데이터

(deploy_ray_func pid=1069) tcmalloc: large alloc 1417158656 bytes == 0x1c6e2000 @  0x7f798d9ca1e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f798a6a3e02 0x7f798a737db6 0x7f798a6aa306 0x7f798a81e6ab 0x7f798a77f88f 0x7f798a8467d3 0x7f798a84772a 0x7f798a85918e 0x7f798a832530 0x7f798aa56f06 0x7f798aa03a3e 0x7f798aa03c96 0x7f798ae9acab 0x7f798ae9bee1
(deploy_ray_func pid=1070) tcmalloc: large alloc 1417158656 bytes == 0x8a890000 @  0x7f88584231e7 0x4a3940 0x5b438c 0x5d0ccd 0x5939af 0x516337 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f88550fce02 0x7f8855190db6 0x7f8855103306 0x7f88552776ab 0x7f88551d888f 0x7f885529f7d3 0x7f88552a072a 0x7f88552b218e 0x7f885528b530 0x7f88554aff06 0x7f885545ca3e 0x7f885545cc96 0x7f88558f3cab 0x7f88558f4ee1
(deploy_ray_func pid=1068) tcmalloc: large alloc 1417158656 bytes == 0x6844000 @  0x7fdeb80c11e7

In [8]:
df=pd.concat([df1, df]) # 2019년 10월, 11월 데이터를 합친다.
del df1 

In [9]:
df["date"]=df["event_time"].map(lambda x: x.split(" ")[0]) # event_time 컬럼을 date와
df["time"]=df["event_time"].map(lambda x: x.split(" ")[1]) # time으로 나누어서 컬럼을 각각 생성
 
del df["event_time"]

In [10]:
def split_category(x):
    if isinstance(x,str):
        category1=x.split(".")[0]
    else: 
        category1=np.nan
    return category1

df["category1"]=df["category_code"].map(lambda x: split_category(x)) # category_code를 "."을 기준으로 나눠서 첫번째 카테고리만 남긴다.
del df["category_code"]

In [11]:
df=df.loc[df["event_type"].isin(["purchase"])].copy() # 구매를 한 사람만 가져온다.

df=df.reset_index(drop=True)

# 나중에 클러스터링을 하기 위해 user,item(product id)리스트를 각각 뽑아둔다.
user_id=df["user_id"].unique() 
product_id=df["product_id"].unique()

#train_df=df[df["date"]<="2019-11-15"].copy()
#test_df=df[df["date"]>"2019-11-15"].copy()

#del df

(raylet) Spilled 6463 MiB, 7 objects, write throughput 203 MiB/s.
(raylet) Spilled 8203 MiB, 18 objects, write throughput 211 MiB/s.


In [12]:
df.head()

,event_type,product_id,category_id,brand,price,user_id,user_session,date,time,category1
0,purchase,1004856,2053013555631882655,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,2019-10-01,00:02:14,electronics
1,purchase,1002532,2053013555631882655,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,2019-10-01,00:04:37,electronics
2,purchase,5100816,2053013553375346967,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,2019-10-01,00:06:02,NaN
3,purchase,13800054,2053013557418656265,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,2019-10-01,00:07:07,furniture
4,purchase,4804055,2053013554658804075,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,2019-10-01,00:09:26,electronics


In [13]:
df["category1"].unique()

array(['electronics', nan, 'furniture', 'appliances', 'auto', 'computers',
       'construction', 'apparel', 'accessories', 'stationery', 'kids',
       'medicine', 'sport', 'country_yard'], dtype=object)

1차 카테고리- electronics와 appliances를 모두 구매한 사람만 모은다. 

### user_item

user(row)와 product_id(columns)로 이루어진 빈칸의 테이블을 만든다.

In [31]:
product=df[["user_id","product_id","time"]].groupby(["user_id","product_id"]).count()

In [32]:
product=product.reset_index(drop=False)

In [33]:
product.head()

,user_id,product_id,time
0,138340325,17301041,1
1,225644257,6800309,1
2,253299396,2400724,1
3,256164170,3900975,1
4,264649825,8500083,1


In [38]:
uLst,pLst=[],[]

for uid in user_id:
    pd_df=product.loc[product["user_id"]==uid]
    uLst.append(uid)
    pd_dict={key:value for key, value in zip(pd_df["product_id"], pd_df["time"])}
    pLst.append(pd_dict)


(raylet) Spilled 16385 MiB, 24671 objects, write throughput 167 MiB/s.


KeyboardInterrupt: ignored

In [ ]:
user_item=pd.DataFrame(data=pLst, index=uLst)

In [ ]:
user_item.fillna(0, inplace=True)

In [ ]:
user_item.head()

In [ ]:
user_item.to_csv("/content/drive/MyDrive/kurly_hackathon/data/clustering_data.csv", encoding="euc-kr")